In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import os

In [4]:
# 8
# qald_path = "/home/kardosp/continuethelist/datasets/QALD/8/data/"
# file_train = "qald-8-train-multilingual.json"
# file_test = "qald-8-test-multilingual.json"

# 7
qald_path = "/home/kardosp/continuethelist/datasets/QALD/7/data/"
file_train = "qald-8-train-multilingual.json"
file_test = "qald-8-test-multilingual.json"

In [5]:
with open(os.path.join(qald_path, file_train), "r") as f:
    train_json_data = json.load(f)

with open(os.path.join(qald_path, file_test), "r") as f:
    test_json_data = json.load(f)

In [6]:
# sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql = SPARQLWrapper("http://localhost:8890/sparql")
# "http://localhost:8890/sparql"
# "http://dbpedia.org/sparql"
sparql.setReturnFormat(JSON)

def run_sparql_query(query):
    sparql.setQuery(query)

    results = None
    try:
        ret = sparql.queryAndConvert()

        for r in ret["results"]["bindings"]:
            if results is None:
                results = [r]
            else:
                results.append(r)
    except Exception as e:
        print("Error -->", e)
    return results

def select_question(q, lang="en"):
    for item in q:
        if item["language"] == lang:
            return item["string"]
    return None

def add_answer_to_questions(input_df):

    questions = list()
    for i in range(len(input_df)):
        print(i)
        result = run_sparql_query(input_df[i]["query"]["sparql"])

        result_extracted = list()
        if result is not None:
            for item in result:
                if "callret-0" in item:
                    result_extracted.append(item["callret-0"]["value"])
                elif "uri" in item:
                    result_extracted.append(item["uri"]["value"])
                else:
                    try:
                        result_extracted.append(item[list(item.keys())[0]]["value"])
                    except Exception as e:
                        print("New enemy encountered: ", item, e)
                # elif "carbs" in item:
                #     result_extracted.append(item["carbs"]["value"])
                # elif "date" in item:
                #     result_extracted.append(item["date"]["value"])
                # elif "c" in item:
                #     result_extracted.append(item["c"]["value"])
                # elif "ni" in item:
                #     result_extracted.append(item["ni"]["value"])
                # elif "num" in item:
                #     result_extracted.append(item["num"]["value"])
                # elif "n" in item:
                #     result_extracted.append(item["n"]["value"])
                # elif "string" in item:
                #     result_extracted.append(item["string"]["value"])
                # elif "x" in item:
                #     result_extracted.append(item["x"]["value"])
                # elif "books" in item:
                #     result_extracted.append(item["books"]["value"])
                # else:
                #     print("New enemy encountered: ", item)

        result_container = dict()
        result_container["result"] = result_extracted
        result_container["_id"] = input_df[i]["id"]
        result_container["corrected_question"] = select_question(input_df[i]["question"])
        result_container["sparql_query"] = input_df[i]["query"]
        result_container["expected_query_result"] = [item["uri"]["value"] for item in train_json_data["questions"][0]["answers"][0]["results"]["bindings"]]

        questions.append(result_container)

    return questions

In [7]:
add_answer_to_questions([train_json_data["questions"][0]])

0


[{'result': ['http://dbpedia.org/resource/Lestat_(musical)',
   'http://dbpedia.org/resource/Aida_(musical)',
   'http://dbpedia.org/resource/Billy_Elliot_the_Musical',
   'http://dbpedia.org/resource/The_Lion_King_(musical)'],
  '_id': '1',
  'corrected_question': 'List all the musicals with music by Elton John.',
  'sparql_query': {'sparql': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE { ?uri rdf:type dbo:Musical ; dbo:musicBy res:Elton_John }'},
  'expected_query_result': ['http://dbpedia.org/resource/The_Lion_King_(musical)',
   'http://dbpedia.org/resource/The_Lion_King_Jr_(musical)',
   'http://dbpedia.org/resource/Billy_Elliot_the_Musical',
   'http://dbpedia.org/resource/Lestat_(musical)',
   'http://dbpedia.org/resource/Aida_(musical)']}]

In [8]:
questions_train = add_answer_to_questions(train_json_data["questions"])
questions_test = add_answer_to_questions(test_json_data["questions"])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Error --> 'results'
17
18
19
20
21
22
23
24
25
Error --> 'results'
26
27
28
29
30
Error --> 'results'
31
32
33
34
35
36
37
38
39
Error --> 'results'
40
41
42
43
44
45
46
Error --> 'results'
47
48
Error --> 'results'
49
50
51
52
Error --> 'results'
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
Error --> 'results'
69
Error --> 'results'
70
71
72
Error --> 'results'
73
Error --> 'results'
74
75
76
77
78
79
80
81
82
83
Error --> 'results'
84
Error --> 'results'
85
86
Error --> 'results'
87
88
89
90
91
Error --> 'results'
92
Error --> 'results'
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
Error --> 'results'
109
110
111
112
113
114
115
116
117
118
Error --> 'results'
119
120
121
Error --> 'results'
122
123
124
125
126
127
128
129
130
131
Error --> 'results'
132
133
134
135
136
137
138
139
140
141
142
Error --> 'results'
143
144
145
146
147
148
149
150
151
Error --> 'results'
152
153
154
Error --> 'results'
155
156
Error --> 'results'
15

In [9]:
with open(os.path.join(qald_path, "qald_dump_w_answers_train.json"), "w") as f:
    json.dump(questions_train, f)

with open(os.path.join(qald_path, "qald_dump_w_answers_test.json"), "w") as f:
    json.dump(questions_test, f)

# Filter questions

In [10]:
with open(os.path.join(qald_path, "qald_dump_w_answers_train.json"), "r") as f:
    questions_train = json.load(f)

with open(os.path.join(qald_path, "qald_dump_w_answers_test.json"), "r") as f:
    questions_test = json.load(f)

In [11]:
valid_questions_train = list()
valid_questions_test = list()

lower_bound = 10
upper_bound = 100

for item in questions_train:
    if upper_bound > len(item["result"]) > lower_bound:
        valid_questions_train.append(item)

for item in questions_test:
    if upper_bound > len(item["result"]) > lower_bound:
        valid_questions_test.append(item)

In [12]:
print(len(valid_questions_train), len(valid_questions_test))

8 0


In [13]:
with open(os.path.join(qald_path, "qald_dump_valid_train.json"), "w") as f:
    json.dump(valid_questions_train, f)

with open(os.path.join(qald_path, "qald_dump_valid_test.json"), "w") as f:
    json.dump(valid_questions_test, f)

In [14]:
from collections import Counter
print(Counter([len(item["result"]) for item in questions_train]))

Counter({1: 132, 0: 34, 2: 15, 3: 10, 4: 4, 7: 4, 10: 3, 32: 2, 5: 2, 8: 2, 12: 1, 6: 1, 13: 1, 16: 1, 196: 1, 9: 1, 34: 1, 11: 1, 30: 1, 117: 1, 2097: 1})
